In [ ]:
"""
Projeto – Big Data com Python, Spark e Pandas

Alunos:
• Italo Rodrigues Bezerra - 202309314363
• Arthur Amorim da Silva – 202408656491
• Gabriel Arcanjo Gonzaga – 202402817973
• João Victor da Silva – 202403676427
• Antônio Luís Alves de Oliveira – 202404250717
• Ian da Nóbrega Araújo – 202402816081
• Antônio Lucas Silva Pinto - 202308418319
• Valber de Lima Pereira da Silva - 202051642289

Dataset escolhido: Arquivos (CSV/JSON) - Segurança Operacional - Ocorrências
Aeronáuticas

"""

In [ ]:
"""
Justificativa da escolha:
A escolha do conjunto de dados sobre ocorrências aeronáuticas se justifica pela sua relevância social, econômica e de segurança, pois a aviação é um setor estratégico
e altamente regulado. A análise desses dados permite identificar padrões de incidentes e fatores de risco em diferentes contextos operacionais.
Além disso, o banco apresenta inconsistências e valores nulos, oferecendo um desafio prático para aplicar técnicas de limpeza, padronização e análise exploratória.
Assim, une importância real para a segurança aérea, valor acadêmico e potencial didático no tratamento de dados.

Descrição dos dados:
O conjunto de dados possui 12.226 registros e 46 colunas (≈4 MB), reunindo informações técnicas, operacionais, geográficas e humanas sobre ocorrências aeronáuticas.
Inclui variáveis categóricas, numéricas, textuais e temporais, como identificação, classificação, data, local, tipo de aeronave, fase de operação e lesões.
Apresenta valores nulos e inconsistências, exigindo limpeza e padronização, o que o torna ideal para análises estatísticas e práticas de tratamento de dados.

"""

In [ ]:
import os

In [ ]:
#Instalar o java

! apt-get install openjdk-17-jdk-headless -qq > /dev/null

In [ ]:
#Baixar, descompactar e configurar o apache spark

! wget -q https://dlcdn.apache.org/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz

In [ ]:
! tar -xf spark-4.0.1-bin-hadoop3.tgz

In [ ]:
! rm -rf spark-4.0.1-bin-hadoop3.tgz

In [ ]:
#Confirgurar as variaveis de ambiente

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ['SPARK_HOME'] = "/content/spark-4.0.1-bin-hadoop3"

In [ ]:
! pip install -q findspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Iniciar spark e criar uma sessão

import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LeituraCSV_Colab") \
    .getOrCreate()

In [ ]:
import csv

# Caminho do arquivo original
input_path = "/content/V_OCORRENCIA_AMPLA.csv"

# Caminho do arquivo limpo que vamos gerar
output_path = "/content/V_OCORRENCIA_AMPLA_limpo.csv"

# Abrir o arquivo original e criar o arquivo limpo
with open(input_path, "r", encoding="utf-8") as infile, \
     open(output_path, "w", encoding="utf-8", newline="") as outfile:

    reader = csv.reader(infile, delimiter=';')
    writer = csv.writer(outfile, delimiter=';')

    for i, row in enumerate(reader):
        if i == 0:
            continue  # pular a primeira linha com "M-oM-;M-?Atualizado..."
        # remover caracteres estranhos de cada célula
        clean_row = [cell.replace("-CM-", "").replace("^M", "").replace("\ufeff", "") for cell in row]
        writer.writerow(clean_row)


In [ ]:
from pyspark.sql import SparkSession

# Criar SparkSession
spark = SparkSession.builder.appName("LeituraCSV_Colab").getOrCreate()

# Ler o CSV limpo
df = spark.read \
    .option("header", True) \
    .option("sep", ";") \
    .option("inferSchema", True) \
    .csv(output_path)

# Mostrar os dados organizados
df.show(10, truncate=False)
df.printSchema()


+--------------------+---------------+----------------------------------+---------------------------+------------------+------------------+----------------------+-------------+------------+-------------------------------------------------+------------------------+--------+---------+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-------

In [ ]:
# Esta consulta seleciona todas as ocorrências do estado de São Paulo (UF = 'SP')
# e exibe as colunas Classificação da Ocorrência, Descrição do Tipo, Fase da Operação, Município e UF.
# O objetivo é visualizar os dados filtrados por estado para análises futuras,
# mantendo todas as colunas inteiras sem cortar os textos longos.

df.createOrReplaceTempView("tabela_temporaria")
spark.sql("""
    SELECT
       Classificacao_da_Ocorrencia AS Classificacao,
       Descricao_do_Tipo AS Tipo,
       Fase_da_Operacao AS Fase,
       Municipio,
       UF AS estado
    FROM tabela_temporaria
    WHERE UF = 'SP'
""").show(20, truncate=False)

+---------------+-------------------------------------------------+------------------+---------------------+------+
|Classificacao  |Tipo                                             |Fase              |Municipio            |estado|
+---------------+-------------------------------------------------+------------------+---------------------+------+
|Acidente       |FALHA OU MAU FUNCIONAMENTO DO MOTOR              |Decolagem         |Tietê                |SP    |
|Acidente       |PERDA DE CONTROLE EM VOO                         |Decolagem         |Tietê                |SP    |
|Acidente       |INDETERMINADO                                    |Indeterminada     |SÃO PEDRO            |SP    |
|Acidente       |COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO|Pouso             |IPATINGA             |SP    |
|Incidente Grave|FALHA OU MAU FUNCIONAMENTO DO MOTOR              |Subida            |SOROCABA             |SP    |
|Incidente Grave|EXCURSÃO DE PISTA                                |NULL 

In [ ]:
# Esta consulta agrupa as ocorrências do estado de São Paulo (UF = 'SP') por Município e Classificação,
# contando quantas ocorrências existem em cada grupo.
# Isso ajuda a identificar municípios com maior número de acidentes ou incidentes.

df.createOrReplaceTempView("tabela_temporaria")
spark.sql("""
    SELECT
        Municipio,
        Classificacao_da_Ocorrencia AS Classificacao,
        COUNT(*) AS total
    FROM tabela_temporaria
    WHERE UF = 'SP'
    GROUP BY Municipio, Classificacao_da_Ocorrencia
    ORDER BY total DESC
""").show(20, truncate=False)

+-------------------+---------------+-----+
|Municipio          |Classificacao  |total|
+-------------------+---------------+-----+
|null               |Acidente       |108  |
|null               |Incidente Grave|34   |
|SÃO PAULO          |Incidente Grave|30   |
|BRAGANÇA PAULISTA  |Acidente       |29   |
|SÃO PAULO          |Acidente       |27   |
|BRAGANÇA PAULISTA  |Incidente Grave|24   |
|ATIBAIA            |Acidente       |17   |
|AMERICANA          |Acidente       |14   |
|JUNDIAÍ            |Acidente       |12   |
|BIRITIBA-MIRIM     |Acidente       |12   |
|CAMPINAS           |Acidente       |11   |
|JUNDIAÍ            |Incidente Grave|11   |
|SOROCABA           |Acidente       |10   |
|BAURU              |Acidente       |9    |
|UBATUBA            |Acidente       |9    |
|GUARULHOS          |Acidente       |8    |
|RIO CLARO          |Acidente       |8    |
|SÃO JOSÉ DOS CAMPOS|Incidente Grave|8    |
|BEBEDOURO          |Acidente       |8    |
|BOITUVA            |Acidente   

In [ ]:
# Esta consulta seleciona todas as ocorrências do estado de São Paulo (UF = 'SP')
# que aconteceram dentro de um período específico, entre 1º de janeiro de 2020 e 31 de dezembro de 2020.
# Ela agrupa os dados por Classificação da Ocorrência e conta quantas ocorrências existem em cada grupo.
# O objetivo é analisar a frequência de cada tipo de ocorrência ao longo do período,
# identificando quais classificações foram mais comuns em SP durante o ano de 2020.


df.createOrReplaceTempView("tabela_temporaria")
spark.sql("""
    SELECT
        Classificacao_da_Ocorrencia AS Classificacao,
        COUNT(*) AS total
    FROM tabela_temporaria
    WHERE UF = 'SP'
       AND Data_da_Ocorrencia BETWEEN '2020-01-01' AND '2020-12-31'
    GROUP BY Classificacao_da_Ocorrencia
    ORDER BY total DESC
""").show(100, truncate=False)

+---------------+-----+
|Classificacao  |total|
+---------------+-----+
|Acidente       |37   |
|Incidente Grave|17   |
+---------------+-----+



In [ ]:
# Esta consulta retorna as linhas da tabela onde a coluna Data_da_Ocorrencia está nula.
# Essas linhas representam ocorrências sem informação de data, o que pode prejudicar análises temporais,
# cálculos de frequência por período ou qualquer estudo relacionado ao momento em que o evento ocorreu.
# É importante identificar esses registros para limpeza, preenchimento ou exclusão antes de análises mais detalhadas.

df.createOrReplaceTempView("tabela_temporaria")

spark.sql("""
    SELECT *
    FROM tabela_temporaria
    WHERE Data_da_Ocorrencia IS NULL
""").show(10, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+--------------------+---------------------------+------------------+------------------+---------+----+------+-----------------+----+--------+---------+-----------------+---------+---------+---------------------+--------+------------------+----------------+--------+----------------+--------------------+-------------------+-------------------------+-------------------------+-----------------------+-------------------------+-------------------------+-----------------------+------------------------+------------------------+----------------------+------------------+------------------+--------------------------------+--------------------------------+------

In [ ]:
# A coluna "Regiao" foi criada a partir da coluna "UF" para classificar cada ocorrência em uma das cinco regiões do Brasil:
# Norte, Nordeste, Centro-Oeste, Sudeste e Sul. Essa coluna permite análises agregadas por região geográfica,
# facilitando a visualização de padrões de ocorrência e distribuição de acidentes/incidentes de forma mais ampla.
# Por exemplo, podemos usar "Regiao" em GROUP BY, filtros e gráficos para comparar a frequência de eventos entre as regiões.


from pyspark.sql.functions import when, col

# Cria uma nova coluna "Regiao" baseada na coluna UF
df = df.withColumn(
    "Regiao",
    when(col("UF").isin("MG","SP","RJ","ES"), "Sudeste")
    .when(col("UF").isin("PR","SC","RS"), "Sul")
    .when(col("UF").isin("MT","MS","GO","DF"), "Centro-Oeste")
    .when(col("UF").isin("BA","SE","AL","PE","PB","RN","CE","PI","MA"), "Nordeste")
    .when(col("UF").isin("AC","RO","RR","AP","PA","AM","TO"), "Norte")
)

display(df)


DataFrame[Numero_da_Ocorrencia: string, Numero_da_Ficha: string, Operador_Padronizado: string, Classificacao_da_Ocorrencia: string, Data_da_Ocorrencia: string, Hora_da_Ocorrencia: string, Municipio: string, UF: string, Regiao: string, Descricao_do_Tipo: string, ICAO: string, Latitude: string, Longitude: string, Tipo_de_Aerodromo: string, Historico: string, Matricula: string, Categoria_da_Aeronave: string, Operador: string, Tipo_de_Ocorrencia: string, Fase_da_Operacao: string, Operacao: string, Danos_a_Aeronave: string, Aerodromo_de_Destino: string, Aerodromo_de_Origem: string, Lesoes_Fatais_Tripulantes: string, Lesoes_Fatais_Passageiros: string, Lesoes_Fatais_Terceiros: string, Lesoes_Graves_Tripulantes: string, Lesoes_Graves_Passageiros: string, Lesoes_Graves_Terceiros: string, Lesoes_Leves_Tripulantes: string, Lesoes_Leves_Passageiros: string, Lesoes_Leves_Terceiros: string, Ilesos_Tripulantes: string, Ilesos_Passageiros: string, Lesoes_Desconhecidas_Tripulantes: string, Lesoes_Desco

In [ ]:
# Esta consulta filtra todas as ocorrências cujo campo "Descricao_do_Tipo" contém a palavra "AVE".
# O objetivo é identificar acidentes ou incidentes relacionados a aves, como colisões com pássaros.
# Essa filtragem permite analisar padrões específicos desse tipo de ocorrência,
# como frequência por UF, Município, Fase da Operação ou Classificação, facilitando estudos de mitigação.


from pyspark.sql.functions import col, lower

# Filtrar ocorrências cujo Descricao_do_Tipo contém "AVE" (case-insensitive)
acidente = df.filter(lower(col("Descricao_do_Tipo")).like("%ave%"))

# Visualizar os dados de forma organizada
acidente.show(50, truncate=False)  # mostra 50 linhas sem cortar colunas


+--------------------+------------------------------+------------------------------------------+---------------------------+------------------+------------------+------------------------+--------+------------+--------------------------------------------------------------------------------------------+-----------------+--------+---------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Converter o DataFrame Spark para pandas
df = df.toPandas()

In [ ]:
#Descobrindo as colunas duplicadas

import pandas as pd

df[df.duplicated(keep=False)]

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
19,27085,202107578,SANTO ANGELO AEROAGRICOLA LTDA,Acidente,2021-02-12,16:00,CONCEIÇÃO DAS ALAGOAS,MG,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,...,null,null,null,PA-36-375,L1P,PA36,2177,1,PIPER AIRCRAFT,verdadeiro
20,27085,202107578,SANTO ANGELO AEROAGRICOLA LTDA,Acidente,2021-02-12,16:00,CONCEIÇÃO DAS ALAGOAS,MG,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,...,null,null,null,PA-36-375,L1P,PA36,2177,1,PIPER AIRCRAFT,verdadeiro
52,"APÓS REALIZAR O PROCEDIMENTO HOTEL 5, A AERONA...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
56,"APÓS REALIZAR O PROCEDIMENTO HOTEL 5, A AERONA...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
299,A aeronave teve danos substanciais.,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4858,40874,202395165533,SAO BENTO AVIACAO AGRICOLA LTDA,Acidente,2023-11-30,10:30:00,PALMEIRA DAS MISSÕES,RS,Sul,EXCURSÃO DE PISTA,...,null,null,null,EMB-201A,L1P,IPAN,1800,1,NEIVA,verdadeiro
4926,33006,2022067658,LUCAS MACHADO SOARES,Acidente,2022-07-19,17:00,LOANDA,PR,Sul,INDETERMINADO,...,null,null,null,R44 II,H1P,R44,1134,4,ROBINSON HELICOPTER,verdadeiro
4927,33006,2022067658,LUCAS MACHADO SOARES,Acidente,2022-07-19,17:00,LOANDA,PR,Sul,INDETERMINADO,...,null,null,null,R44 II,H1P,R44,1134,4,ROBINSON HELICOPTER,verdadeiro
5163,28704,202124302,QNE ESCOLA DE AVIACAO CIVIL LTDA - ME,Acidente,2021-07-04,13:32,GUARAPARI,ES,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,...,null,null,null,PA-34-200,L2P,PA34,1905,7,PIPER AIRCRAFT,verdadeiro


In [ ]:
#Excluindo duplicados

df.drop_duplicates(inplace=True)

In [ ]:
#Descobrindo se ainda existe duplicados

df[df.duplicated(keep=False)]

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO


In [ ]:
#Excluindo colunas irrelevantes

colunas_para_excluir = [
    "Aerodromo", "Latitude", "Longitude", "Codigo_ocorrencia1",
    "Codigo_ocorrencia2", "Codigo_ocorrencia3", "Codigo_da_Aeronave",
    "Aeronave_Matricula", "Codigo_da_Ocorrencia2", "Codigo_da_Ocorrencia3",
    "ICAO", "Tipo_de_Aeronave", "Fator_Contribuinte", "Area", "Localizacao",
    "UF_Cidade", "Situacao_da_Ocorrencia", "Relatorio_Final", "Boletim",
    "Numero_da_Ocorrencia", "Data_da_Publicacao", "Data_do_Relatorio"

]

df.drop(columns=[c for c in colunas_para_excluir if c in df.columns], inplace=True)


In [ ]:
#Descobrindo a quantidade de nulos por coluna

df.isnull().sum()

,0
Numero_da_Ficha,172
Operador_Padronizado,182
Classificacao_da_Ocorrencia,178
Data_da_Ocorrencia,176
Hora_da_Ocorrencia,176
Municipio,176
UF,176
Regiao,951
Descricao_do_Tipo,195
Tipo_de_Aerodromo,176


In [ ]:
#Substituindo valores nulos por "Não informado" em colunas tipo string

colunas_nao_numericas = [
    'Numero_da_Ficha', 'Operador_Padronizado', 'Classificacao_da_Ocorrencia',
    'Data_da_Ocorrencia', 'Hora_da_Ocorrencia', 'Municipio', 'UF', 'Regiao',
    'Descricao_do_Tipo', 'Tipo_de_Aerodromo', 'Historico', 'Matricula',
    'Categoria_da_Aeronave', 'Operador', 'Tipo_de_Ocorrencia',
    'Fase_da_Operacao', 'Operacao', 'Danos_a_Aeronave',
    'Aerodromo_de_Destino', 'Aerodromo_de_Origem', 'Modelo', 'CLS',
    'Tipo_ICAO', 'Nome_do_Fabricante', 'PSSO'
]

for coluna in colunas_nao_numericas:
    if coluna in df.columns:

        # 1. Substitui nulos por "Não informado"
        df[coluna] = df[coluna].fillna("Não informado")

        # 2. Substitui valores vazios por "Não informado"
        df[coluna] = df[coluna].astype(str).str.strip().replace("", "Não informado")



In [ ]:
# Substituir valores nulos por 0 nas colunas numéricas
colunas_numericas = [
    'Lesoes_Fatais_Tripulantes',
    'Lesoes_Fatais_Passageiros',
    'Lesoes_Fatais_Terceiros',
    'Lesoes_Graves_Tripulantes',
    'Lesoes_Graves_Passageiros',
    'Lesoes_Graves_Terceiros',
    'Lesoes_Leves_Tripulantes',
    'Lesoes_Leves_Passageiros',
    'Lesoes_Leves_Terceiros',
    'Ilesos_Tripulantes',
    'Ilesos_Passageiros',
    'Lesoes_Desconhecidas_Tripulantes',
    'Lesoes_Desconhecidas_Passageiros',
    'Lesoes_Desconhecidas_Terceiros',
    'PMD',
    'Numero_de_Assentos'
]

for coluna in colunas_numericas:
    if coluna in df.columns:
        df[coluna] = df[coluna].fillna(0)

# Conferir se ainda existem valores nulos
df[colunas_numericas].isna().sum()


,0
Lesoes_Fatais_Tripulantes,0
Lesoes_Fatais_Passageiros,0
Lesoes_Fatais_Terceiros,0
Lesoes_Graves_Tripulantes,0
Lesoes_Graves_Passageiros,0
Lesoes_Graves_Terceiros,0
Lesoes_Leves_Tripulantes,0
Lesoes_Leves_Passageiros,0
Lesoes_Leves_Terceiros,0
Ilesos_Tripulantes,0


In [ ]:
# O DataFrame ainda tem strings literalmente iguais a "null", isso não são valores nulos do pandas,
# são apenas texto. Então aqui esta sendo excluido esses textos indesejados
valores_invalidos = ["null", "NULL", "Null", "nan", "NaN", "None", ""]

for coluna in df.columns:
    df = df[~df[coluna].astype(str).str.strip().isin(valores_invalidos)]


In [ ]:
#Conferindo as mudanças exibindo novamente ocorrências no estado de SP, só que agora em pandas
#loc

df_sp = df.loc[df["UF"] == "SP", [
    "Classificacao_da_Ocorrencia",
    "Descricao_do_Tipo",
    "Fase_da_Operacao",
    "Municipio",
    "UF"
]]

df_sp.head(10)


,Classificacao_da_Ocorrencia,Descricao_do_Tipo,Fase_da_Operacao,Municipio,UF
285,Incidente Grave,PERDA DE CONTROLE NO SOLO,Decolagem,BRAGANÇA PAULISTA,SP
286,Acidente,PERDA DE CONTROLE NO SOLO,Pouso,RIO CLARO,SP
287,Acidente,FALHA OU MAU FUNCIONAMENTO DO MOTOR,Em rota,GUARUJÁ,SP
337,Acidente,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,Em rota,BAURU,SP
338,Incidente Grave,FALHA OU MAU FUNCIONAMENTO DO MOTOR,Decolagem,ITÁPOLIS,SP
346,Acidente,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,Pouso,VOTUPORANGA,SP
391,Acidente,FALHA OU MAU FUNCIONAMENTO DO MOTOR,Em rota,BIRIGUI,SP
1100,Acidente,PERDA DE CONTROLE EM VOO,Manobra,LEME,SP
1101,Acidente,OPERAÇÃO A BAIXA ALTITUDE,Manobra,LEME,SP
1102,Acidente,PERDA DE CONTROLE EM VOO,Em rota,SÃO PAULO,SP


In [ ]:
#Pegar as 4 primeiras linhas e colunas específicas
#Iloc

df.iloc[:4, [0, 4, 7, 8]]

,Numero_da_Ficha,Hora_da_Ocorrencia,Regiao,Descricao_do_Tipo
74,PT-WOF 05MAI2020 20H46 SERIPA2,20:46:00,Nordeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...
76,PU-DGH 21ABR2020 18H00 SERIPA3,18:00:00,Sudeste,MANOBRA ABRUPTA
77,PU-DGH 21ABR2020 18H00 SERIPA3,18:00:00,Sudeste,PERDA DE CONTROLE EM VOO
262,202050591,12:30:00,Norte,EXCURSÃO DE PISTA


In [ ]:
#Filtrando dados pelo estado de SP e tipo de operação
#query

df.query("UF == 'SP' and Classificacao_da_Ocorrencia == 'Acidente'").head(10)

,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,Tipo_de_Aerodromo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
286,PP-FLF 05MAR2020 20H00 SERIPA4,AEROCLUBE DE RIO CLARO,Acidente,2020-03-05,20:00:00,RIO CLARO,SP,Sudeste,PERDA DE CONTROLE NO SOLO,Público,...,0,0,0,AB-115,L1P,AB11,770,3,AERO BOERO,verdadeiro
287,202087125,VISUAL PROPAGANDA AEREA LTDA-EPP,Acidente,2020-02-25,13:50:00,GUARUJÁ,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,-,...,0,0,0,170A,L1P,C170,862,1,CESSNA AIRCRAFT,verdadeiro
337,PT-PAZ 10MAR2020 16H30 SERIPA4,AEROCLUBE DE BAURU,Acidente,2020-03-10,16:30:00,BAURU,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,-,...,0,0,0,LK-10A,L00,GLID,397,2,LAISTER-KAUFFMAN,verdadeiro
346,202031290,AERO CLUBE DE VOTUPORANGA,Acidente,2020-05-14,21:40:00,VOTUPORANGA,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,Público,...,0,0,0,95-B55,L2P,BE55,2268,4,BEECH AIRCRAFT,verdadeiro
391,202032172,LOURIVAL FORTUNA,Acidente,2020-04-18,20:00:00,BIRIGUI,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,-,...,0,0,0,EMB-711C,L1P,P28R,1202,4,NEIVA,verdadeiro
1100,202046553,MARCELO MENDES FERNANDES,Acidente,2020-01-25,17:00:00,LEME,SP,Sudeste,PERDA DE CONTROLE EM VOO,Público,...,0,0,0,ML450T,L1P,ULAC,390,2,MICROLEVE,verdadeiro
1101,202046553,MARCELO MENDES FERNANDES,Acidente,2020-01-25,17:00:00,LEME,SP,Sudeste,OPERAÇÃO A BAIXA ALTITUDE,Público,...,0,0,0,ML450T,L1P,ULAC,390,2,MICROLEVE,verdadeiro
1102,202011216,IZAURA TETE TEODORO,Acidente,2020-01-16,12:00:00,SÃO PAULO,SP,Sudeste,PERDA DE CONTROLE EM VOO,-,...,0,0,0,R44,H1P,R44,1089,4,ROBINSON HELICOPTER,verdadeiro
1117,PR-RMZ 01JAN2019 12H55 SERIPA4,HELICON TAXI AEREO LTDA,Acidente,2019-01-01,12:55:00,UBATUBA,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,-,...,0,0,0,R44 II,H1P,R44,1134,4,ROBINSON HELICOPTER,verdadeiro
1165,202077266,AEROMAJ AVIACAO AGRICOLA LTDA,Acidente,2020-01-03,21:15:00,LEME,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,-,...,0,0,0,EMB-202A,L1P,IPAN,1800,1,NEIVA,verdadeiro


In [ ]:
#Top 10 estados com mais ocorrências
#Groupby

df.groupby("UF")["Numero_da_Ficha"].count().sort_values(ascending=False).head(10)


,Numero_da_Ficha
UF,
Não informado,176
SP,67
Substancial,33
Leve,30
MG,29
MT,23
PR,23
GO,18
RS,17


In [ ]:
# Removendo estados errados
df = df[~df["UF"].isin(["Substancial", "Leve"])]

In [ ]:
# Estatistica da classificação das ocorrencias
# A coluna possui 5.292 registros e 95 categorias diferentes, mas é fortemente dominada pela
# classificação “Acidente”, que representa cerca de 70% das ocorrências.
# describe()

df[["Classificacao_da_Ocorrencia"]].describe()

,Classificacao_da_Ocorrencia
count,539
unique,76
top,Acidente
freq,231


In [ ]:
# Ranking das regiões com mais ocorrências
# value_counts()

df["Regiao"].value_counts().head(10)


,count
Regiao,
Não informado,260
Sudeste,116
Centro-Oeste,53
Sul,52
Norte,38
Nordeste,20


In [ ]:
# Gráfico de barras (bar) que mostra o número de ocorrências por estado

import plotly.express as px
uf_counts = df_filtrado['UF'].value_counts().reset_index()
uf_counts.columns = ["UF", "Count"]

fig = px.bar(
    uf_counts,
    x="UF",
    y="Count",
    color="UF",
    text="Count",
    color_discrete_sequence=px.colors.qualitative.Prism,
    title="Ocorrências por Estado (UF)"
)

fig.update_traces(
    texttemplate='%{text}',
    textposition='outside'
)

fig.update_layout(
    xaxis_title="",
    yaxis_title=""
)

fig.show()


In [ ]:
# Gráfico de pizza (pie) que mostra a proporção das 3 ocorrências mais comuns

import plotly.express as px

top10 = (
    df['Classificacao_da_Ocorrencia']
    .value_counts()
    .head(3)
    .reset_index()
)

top10.columns = ["Classificacao", "Count"]

# Gráfico de pizza
fig = px.pie(
    top10,
    names="Classificacao",
    values="Count",
    color="Classificacao",
    color_discrete_sequence=px.colors.qualitative.Set3,
    title="Top 3 Classificações de Ocorrência (Pie Chart)"
)

fig.show()




In [ ]:
# Gráfico de linha (line) em que mostra a evolução de Ocorrências entre 2015 e 2020

import plotly.express as px

fig = px.line(
    contagem_mensal,
    x="Mes",
    y="Count",
    markers=True,
    title="Ocorrências entre 2015 e 2020 (por mês)",
    color_discrete_sequence=["#1f77b4"]
)

fig.update_layout(xaxis_title="", yaxis_title="")
fig.show()


In [ ]:
#Gráfico de Histograma (histogram) que mostra a quantidade de ocorrências por ano

import pandas as pd
import plotly.express as px


df['Data_da_Ocorrencia'] = pd.to_datetime(df['Data_da_Ocorrencia'], errors='coerce')

df['Ano'] = df['Data_da_Ocorrencia'].dt.year.astype('Int64')

# Criar histograma
fig = px.histogram(
    df,
    x="Ano",
    title="Histograma de Ocorrências por Ano)",
    nbins=50
)

fig.update_layout(
    xaxis=dict(type="linear"),
    xaxis_title="Ano",
    yaxis_title="Número de Ocorrências"
)

fig.show()


In [ ]:
#Gráfico de dispersão (scatter) que mostra as ocorrências por estado da região nordeste


fig = px.scatter(
    uf_counts_ne,
    x="UF",
    y="Count",
    color="UF",
    size="Count",
    title="Ocorrências por Estado – Região Nordeste"
)

fig.update_layout(xaxis_title="", yaxis_title="")
fig.show()


In [ ]:
"""
Conclusão geral:

A análise das ocorrências aeronáuticas permitiu identificar padrões importantes relacionados à segurança da aviação no Brasil.
Observou-se que o estado de São Paulo concentra a maior parte dos registros, refletindo seu volume operacional. A classificação “Acidente” domina
amplamente o dataset, indicando a necessidade de atenção contínua a fatores de risco recorrentes. Em termos temporais, o ano de 2019 apresentou o
maior número de ocorrências, sugerindo um período de maior instabilidade ou aumento na fiscalização e registro. A análise regional destacou ainda que,
no Nordeste, o Ceará foi o estado com mais ocorrências, seguido por Bahia e Pernambuco.De forma geral, o estudo demonstrou como técnicas de limpeza,
transformação e visualização de dados podem revelar padrões relevantes para a segurança operacional. Além de aprofundar o entendimento sobre os fatores
envolvidos nos incidentes, o projeto reforçou a importância de dados bem estruturados para apoiar decisões e prevenir novos eventos.


"""